<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install textblob

In [2]:
import numpy as np
import pandas as pd

import re
from textblob import TextBlob, Word
import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score 

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
df['toxic'].sum()

16186

In [9]:
def clear_text(text):
    '''
    Функция на вход принимает текст, возвращает текст,
    очищенный от всех символов, кроме пробелов и букв английского  алфавита
    '''
    text_re = re.sub(r'[^a-zA-Z ]', ' ', text)
    text_split = text_re.split()
    text_join = ' '.join(text_split)
    return text_join

In [10]:
# очистим текст от лишних символов
df['clear_text'] = df['text'].apply(lambda x: clear_text(x))

In [11]:
def lemmatize_with_postag(sentence):
    '''
    Функция на вход принимает текст, возвращает лемматизированный текст
    '''
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [12]:
# лемматизируем текст
df['lemm_text'] = df['clear_text'].apply(lambda x: lemmatize_with_postag(x))

In [13]:
# создадим переменные с признаками и целевым признаком
X = df['lemm_text']
y = df['toxic']

In [15]:
# разделим  выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, y_train.shape

((143362,), (143362,))

Мы загрузили данные. Всего в нашем распоряжении 159292 объекта, пропусков нет. Выборка несбалансирована, в положительном классе 10% выборки.

Мы очистили текст от  личшних символов (оставили буквы английского алфавита и пробелы), лемматизировали текст, создали  переменные с признаками и целевым признаком. Разделили выборку на тренировочную  и тестовую, размер тестовой части - 10%.

## Обучение

In [18]:
# инициализируем трансформер для вычисления TD-IDF
stopwords = set(nltk_stopwords.words('english'))
transform = TfidfVectorizer(stop_words=stopwords)

In [19]:
from sklearn.naive_bayes import ComplementNB

In [20]:
# зададим параметры линейной регресии
lr_opt_params= { 
    'lr_classifier__C': range(1,15),   
    'lr_classifier__class_weight': ['balanced','None'] 
    }

# зададим параметры градиентного бустинга
boost_opt_params= { 
    'boost_classifier__n_estimators': range(10,101,10),
    'boost_classifier__max_depth' : range(1,13)
    }

# зададим параметры стохастического градиентного спуска
sgd_opt_params= {
    'sgd_classifier__alpha': (0.00001, 0.000001),
    'sgd_classifier__penalty': ('l2', 'elasticnet'),
    'sgd_classifier__n_iter_no_change': range(3, 10)
    }

# зададим параметры наивного байеса
bayes_opt_params= {
     'bayes_classifier__alpha': np.logspace(-6, 6, 13)
     }

# инициализируем модели
clfs_params = {
    'lr_classifier': LogisticRegression(max_iter=500),
    'boost_classifier': LGBMClassifier(random_state=42),
    'sgd_classifier': SGDClassifier(random_state=42, class_weight='balanced'),
    'bayes_classifier': ComplementNB()
    }

In [21]:
# создадим функцию для подбора модели
def select_best_model():
    best_score = 0
    i = 0
    opt_param = [lr_opt_params, boost_opt_params, sgd_opt_params, bayes_opt_params]
    for key,val in clfs_params.items():
        
        clf = Pipeline(
            [
                ('transformer', transform),
                (key, val)
            ]
        )
        clf_rand_search = RandomizedSearchCV(clf, opt_param[i], cv=5, scoring='f1')
        clf_rand_search.fit(X_train, y_train)
        
        print('Best {} parameters:'.format(key))
        print(clf_rand_search.best_params_)
        
        print('Best {} score: {}'.format(key,clf_rand_search.best_score_))
        
        if  clf_rand_search.best_score_> best_score:
            
            best_score=clf_rand_search.best_score_
            best_model=clf_rand_search.best_estimator_
            
        i+=1
    print('-----------------------------------------------------')
     
    print("Best Classifier:\n{} ".format(best_model))
    print("Best Score: {} ".format(best_score))
    return best_model,best_score


result = select_best_model()
best_model, best_score = result[0], result[1]

Best lr_classifier parameters:
{'lr_classifier__class_weight': 'None', 'lr_classifier__C': 13}
Best lr_classifier score: 0.7709513080144632
Best boost_classifier parameters:
{'boost_classifier__n_estimators': 80, 'boost_classifier__max_depth': 12}
Best boost_classifier score: 0.6843447838410496
Best sgd_classifier parameters:
{'sgd_classifier__penalty': 'elasticnet', 'sgd_classifier__n_iter_no_change': 7, 'sgd_classifier__alpha': 1e-05}
Best sgd_classifier score: 0.7420282667046277
Best bayes_classifier parameters:
{'bayes_classifier__alpha': 0.1}
Best bayes_classifier score: 0.6546830626331073
-----------------------------------------------------
Best Classifier:
Pipeline(steps=[('transformer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't"

Мы  обучили следующие модели:
- логистическая регрессия,
- градиентный бустинг LightGBM,
- стохастический градиентный спуск из библиотеки sklearn,
- модель на основе наивного байесовского  алгоритма для несбалансированных данных.

Наилучший результат показала логистическая регрессия, метрика F1 равна 0.77.

## Проверка модели

In [25]:
pred = best_model.predict(X_test)
f1 = f1_score(y_test, pred)
print('Метрика F1 модели:', f1)

Метрика F1 модели: 0.7802690582959642


Мы проверили лучшую модель на тестовой выборке, метрика F1 равна 0.78.

## Вывод

Целью нашей работы было создание и обучение модели классифицировать комментариев пользователей на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности комментариев.

Метрикой качества является F1, целевое значение метрики - не меньше 0.75. 

Всего в нашем распоряжении 159292 объекта, пропусков нет. Выборка несбалансирована, в положительном классе 10% выборки.

Мы очистили текст от  личшних символов (оставили буквы английского алфавита и пробелы), лемматизировали текст, создали  переменные с признаками и целевым признаком. Разделили выборку на тренировочную  и тестовую, размер тестовой части - 10%.

Мы вычислили TF-IDF.

Мы  обучили следующие модели:
- логистическая регрессия,
- градиентный бустинг LightGBM,
- стохастический градиентный спуск из библиотеки sklearn,
- модель на основе наивного байесовского  алгоритма для несбалансированных данных.

Наилучший результат показала логистическая регрессия, метрика F1 равна 0.77.

Мы проверили лучшую модель на тестовой выборке, метрика F1 равна 0.78.

В итоге в качестве модели у нас есть модель <b>логистической регрессии</b>, ее <b>метрика F1 равна 0.78</b>, что выше целевого значения.